# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1695508180000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-08-23&endtime=2023-09-22',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 10131}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [9]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.3,
  'place': '63 km WSW of Cantwell, Alaska',
  'time': 1695340690356,
  'updated': 1695340815461,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023c4zx1m3',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023c4zx1m3&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 26,
  'net': 'ak',
  'code': '023c4zx1m3',
  'ids': ',ak023c4zx1m3,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.54,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.3 - 63 km WSW of Cantwell, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-150.0719, 63.1273, 72.2]},
 'id': 'ak023c4zx1m3'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.30,"63 km WSW of Cantwell, Alaska",1695340690356,1695340815461,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023c4zx1m3,",",ak,",",origin,phase-data,",NaN,NaN,0.54,NaN,ml,earthquake,"M 1.3 - 63 km WSW of Cantwell, Alaska"
1,0.64,"1 km ESE of Devore, CA",1695340522350,1695342375384,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40570136,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",9.0,0.02404,0.10,105.0,ml,earthquake,"M 0.6 - 1 km ESE of Devore, CA"
2,2.00,"9 km WNW of Longbranch, Washington",1695339722340,1695344466470,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uw61961396,",",uw,",",origin,phase-data,",35.0,0.08314,0.23,45.0,ml,earthquake,"M 2.0 - 9 km WNW of Longbranch, Washington"
3,0.94,Idaho-Montana border region,1695339206570,1695392918810,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uu60550642,",",uu,",",origin,phase-data,",11.0,0.01250,0.15,88.0,ml,earthquake,M 0.9 - Idaho-Montana border region
4,0.38,"13 km SW of False Pass, Alaska",1695338978080,1695346766610,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91116358,",",av,",",origin,phase-data,",4.0,NaN,0.06,176.0,ml,earthquake,"M 0.4 - 13 km SW of False Pass, Alaska"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>